<a href="https://colab.research.google.com/github/junity233/MokeSpeech/blob/main/Moke_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.初始化运行环境

运行完了最后会有一行红字，点它下面的那个"RESTART RUNTIME"按钮就可以了

In [ ]:
%pip install common dual tight data prox
%pip install paddlepaddle-gpu==2.3.2.post111 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
%pip install paddlespeech
%pip install yacs
%pip install opencv_python

# 2.下载模型

请根据以下步骤执行：
1. 在Google Drive上新建一个文件夹，命名为 "MokeSpeech"
2. 复制下面链接中的内容到该文件夹内
3. 运行这个代码块的内容

https://drive.google.com/drive/folders/1si3mH-PPwFGc6H5KGfDw6uxCwdEnGbXC?usp=sharing

In [ ]:
from google.colab import drive
drive.mount("/content/gdriver")

%mkdir model
%cp -r /content/gdriver/MyDrive/MokeSpeech/* /content/model

Mounted at /content/gdriver


# 3.加载配置

In [ ]:
from os.path import abspath
import os.path

vocoders = ["hifigan_aishell3", "pwgan_aishell3", "wavernn_csmsc"]

def get_vocoder(name):
    if name not in vocoders:
        return None

    return {
        "model_dir": os.path.abspath("model/voc_model/"+name),
        "model_file": f"{name}.pdmodel",
        "params_file": f"{name}.pdiparams"
    }


am = {
    "model_dir": os.path.abspath("model/am_model"),
    "model_file": f"fastspeech2_mix.pdmodel",
    "params_file": f"fastspeech2_mix.pdiparams"
}

phone_id_map_file = os.path.abspath("model/am_model/phone_id_map.txt")

%mkdir output


# 4.加载模型

In [ ]:
from paddlespeech.t2s.exps.syn_utils import get_am_output
from paddlespeech.t2s.exps.syn_utils import get_frontend
from paddlespeech.t2s.exps.syn_utils import get_predictor
from paddlespeech.t2s.exps.syn_utils import get_voc_output

print("Loading frontend")
frontend = get_frontend(
    lang='mix', phones_dict=phone_id_map_file, tones_dict=None)

print("Loading am")
am_predictor = get_predictor(device="gpu", **am)

print("Loading vocoders...")
voc_predictors = {}

for i in vocoders:
    voc_predictors[i] = get_predictor(
        device="gpu", **get_vocoder(i))
    print(f"\tVocader {i} loaded.")


# 5.Happy Hacking
点击运行即可生成音频

模型解释如下：

0. hifigan_aishell3:速度快，质量低
1. pwgan_aishell3:速度与质量中等
2. wavernn_csmsc:速度慢，质量高

生成的音频存储在output文件夹中，按照时间命名

In [ ]:
import soundfile as sf
import datetime

text=input("输入文本:")
for i,v in enumerate(vocoders):
  print(f"{i}.{v}")

index=int(input("选择模型:"))
if index<0 or index>=len(voc_predictors):
  print("输入有误!")
else:
  voc_predictor=voc_predictors[vocoders[index]]
  am_res = get_am_output(input=text,
                           am_predictor=am_predictor,
                           am="fastspeech2_mix",
                           frontend=frontend,
                           lang="mix",
                           speaker_dict=phone_id_map_file,
                           )

  res = get_voc_output(voc_predictor, am_res)
  now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  file_path=f"output/{now}.wav"
  sf.write(file_path,res,24000)
  print("生成成功!")



# 6.在线播放

In [ ]:
import IPython
IPython.display.Audio(file_path)